In [2]:
import numpy as np
from qutip import *
%load_ext line_profiler
%load_ext Cython

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


# Python

In [ ]:
def py_contract(A, dims_A, inds_I, C_list, inds_C):
    
    '''
    Partial trace of indices inds_I + contraction of inds_C of A with operators in C_list
    '''
    
    # Shape of A
    shape_A = A.shape[0]
    
    dims_I = [dims_A[i] for i in inds_I]
    dims_C = [dims_A[i] for i in inds_C]
    
    # Properties of the state to remain
    inds_B = [i for i in range(len(dims_A)) if i not in inds_I + inds_C]
    dims_B = [dims_A[i] for i in inds_B]
    shape_B = np.prod([1]+dims_B)
    data_B = np.zeros([shape_B, shape_B], dtype = complex)
    
    # Map to convert a multyindex the to the index of the matrix
    unrav_map_A = unrav_map(dims_A)
    
    # Indices in Y axis of the elemenets in the matrix to sum to get the element with index 0 in matrix B
    # [y0, y1, y2, ...]
    Y_sum_0 = get_ellements_1(dims_C + dims_I, inds_C + inds_I, unrav_map_A)

    # Lists of indices in X axis corresponded to Y_sum_0 to sum to get the element with index 0 in matrix B
    # with coefficients {c} to multiply represented contraction with C_list
    # [ [[x00, x01, ...],[c00, c01, ...]], [[x10, x11, ...],[c10, c11, ...]]...]
    # this meanse that
    # B[0,0] = sum_i{  sum_j{A[xij,yi]*cj}  }
    X_sum_0 = get_ellements_2(dims_I, inds_I, dims_C, inds_C, C_list, unrav_map_A)
    coefs_list = [X[1] for X in X_sum_0]
    
    N_sum = len(X_sum_0)
    N_sum_list = range(N_sum)
    B_inds_list = range(shape_B)
    
    # To get other elmenets of B we shift all indices to a required constant.
    X_sum_list = []
    Y_sum_list = []
    for ind_B in B_inds_list:
        B_multind = list(np.unravel_index(ind_B, dims_B))
        ind_A = sum(B_multind[mm]*unrav_map_A[jj] for mm, jj in enumerate(inds_B))
        
        Y_sum_list += [[Y + ind_A for Y in Y_sum_0]]
        X_sum_list += [[[x + ind_A for x in X[0]] for X in X_sum_0]]
    
#     N_X_sum = len(X_sum_list[0][0])
    
    # Calculation of indices for upper triangle + diagonal
    for y_B in B_inds_list:
        # y inds to sum in A for y_B index of B
        y_inds_A = Y_sum_list[y_B]
        
        for x_B in B_inds_list[y_B:]:
            
            # list of x inds corresponed to y inds in y_inds_A
            x_inds_A_list = X_sum_list[x_B]
            
            # Sum through all indices of A to get B[y_B, x_B]
            for y_A in range(len(y_inds_A)):
                
                y_ind = y_inds_A[y_A]
                x_inds_list = x_inds_A_list[y_A]
                coefs = coefs_list[y_A]
                for x_A in range(len(x_inds_list)):
                    data_B[y_B, x_B] += A[y_ind, x_inds_list[x_A]] * coefs[x_A]
    
    
    # Calculation of indices for lower triangle
    for y_B in B_inds_list:
        for x_B in B_inds_list[y_B:]:
            data_B[x_B, y_B] = np.conj(data_B[y_B, x_B])
            
    return data_B, dims_B


def unrav_map(dims):
    map_list = [1]
    for d in dims[1:][::-1]:
        map_list = [map_list[0]*d] + map_list
    
    return map_list

def get_ellements_2(dims_I, inds_I, dims_C, inds_C, C_list, unrav_map_A):
    
    el_0 = [[0], [1]]
    inds_el_A = [el_0]
    for i in range(len(inds_C)):
        C = C_list[i]
        inds_el_A_new = []
        d = dims_C[i]
        k_unrav_list = [k*unrav_map_A[inds_C[i]] for k in range(d)]
        for k1 in range(d):
            for el in inds_el_A:
                el_new_inds = [el_ind + k_add for el_ind in el[0] for k_add in k_unrav_list]
                el_new_coefs = [el_coef*C[k2, k1] for el_coef in el[1] for k2 in range(d)]
                inds_el_A_new += [[el_new_inds, el_new_coefs]]
        inds_el_A = inds_el_A_new
    
    for i in range(len(inds_I)):
        inds_el_A_new = []
        d = dims_I[i]
        for k in range(d):
            k_unrav = k*unrav_map_A[inds_I[i]]
            for el in inds_el_A:
                el_new_inds = [el_ind + k_unrav for el_ind in el[0]]
                inds_el_A_new += [[el_new_inds, el[1]]]
        inds_el_A = inds_el_A_new
    
    return inds_el_A


def get_ellements_1(dims_T, inds_T, unrav_map_A):
    
    inds_el_A = [0]
    for i in range(len(inds_T)):
        inds_el_A_new = []
        
        dt = dims_T[i]
        for k in range(dt):
            k_unrav = k*unrav_map_A[inds_T[i]]
            for j in range(len(inds_el_A)):
                inds_el_A_new += [inds_el_A[j] + k_unrav]
        
        inds_el_A = inds_el_A_new
        
    return inds_el_A

# Cython

In [4]:
%%cython
cimport cython
import numpy as np
cimport numpy as np

cdef extern from "complex.h":
    double complex conj(double complex x)
    
@cython.boundscheck(False)
@cython.cdivision(True)        
cdef void cy_unravel_inds( int ind, int[::1] dims, int * multy_inds ) nogil:
    
    cdef size_t j
    for j from dims.shape[0] > j >= 0:
        
        multy_inds[j] = ind % dims[j]
        ind = ind / dims[j]

cdef int[::1] cy_unrav_map(int[::1] dims):
    cdef int[::1] map_ar = np.ones(dims.shape[0], dtype=np.int32)
    
    cdef size_t j
    for j from dims.shape[0] > j >= 1:
        map_ar[j-1] = map_ar[j]*dims[j]
    
    return map_ar

@cython.boundscheck(False)
@cython.wraparound(False)
cdef int cy_in(int val, int[::1] vec) nogil:
    # val in vec in pure cython
    cdef size_t ii
    for ii from 0 <= ii < vec.shape[0]:
        if val == vec[ii]:
            return 1
    return 0

@cython.boundscheck(False)
@cython.wraparound(False)
cdef void cy_get_ellements_3(int *dims_I, int[::1] inds_I, int *dims_C, int[::1] inds_C, 
                      complex[:,:,::1] C_list, int[::1] unrav_map_A, size_t shape_C, size_t shape_I, 
                       complex[:,::1] coefs_list, int[:,::1] inds_A) nogil:
    
#     cdef size_t n_x, n_A, D, i, k, k1, k2, j, m, c1, c2, n_0, n_1 
#     cdef int d_dim
    cdef size_t n_x, n_A, D, i, k, k1, k2, j, m, c1, c2, n_0, n_1 
    cdef int d_dim
    
    n_x = 1
    n_A = 1
    for i from 0 <= i < inds_C.shape[0]:
        D = dims_C[i]
        d_dim = unrav_map_A[inds_C[i]]
        for k from 1 <= k < D:
            n_0 = (k-1)*n_A
            n_1 = k*n_A
            for j from 0 <= j < n_A:
                inds_A[0][n_1 + j] = inds_A[0][n_0 + j] + d_dim
        
        n_A = n_A*D

        for k1 from D > k1 >= 0:
            for c1 from n_x > c1 >= 0:
                for c2 from n_x > c2 >= 0:
                    for k2 from D > k2 >= 0:
                        coefs_list[k1*n_x+c1][k2*n_x + c2] = coefs_list[c1][c2] * C_list[i, k2, k1]
                        
        n_x = D*n_x
        
    n_A = 1
    n_1 = 1
    for i from 0 <= i < inds_I.shape[0]:
        D = dims_I[i]
        for k from 1 <= k < D:
            d_dim = k*unrav_map_A[inds_I[i]]
            for j from 0 <= j < n_A:
                for m from 0 <= m < shape_C:
                    inds_A[n_1][m] = inds_A[j][m] + d_dim
                n_1 += 1
        n_A = n_1

from libc.stdlib cimport malloc, free
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef cy_contract(complex[:,::1] A, int[::1] dims_A, int[::1] inds_I, complex[:,:,::1] C_list, int[::1] inds_C):
    
    '''
    Partial trace of indices inds_I + contraction of inds_C of A with operators in C_list
    '''
    
    cdef size_t shape_A, shape_B, shape_C, shape_I, i, j, y_B, x_B, i_y_A, i_x_A, n, ind_A, y_A, x_A
    
    # Shape of A
    shape_A = A.shape[0]
    cdef int *dims_C = <int *> malloc(inds_C.shape[0] * sizeof(int))
    cdef int *dims_I = <int *> malloc(inds_I.shape[0] * sizeof(int))
    cdef int *dims_CI = <int *> malloc((inds_C.shape[0] + inds_I.shape[0]) * sizeof(int))
    cdef int *inds_CI = <int *> malloc((inds_C.shape[0] + inds_I.shape[0]) * sizeof(int))

    for i from 0 <= i < inds_C.shape[0]:
        dims_C[i] = dims_A[inds_C[i]]
        dims_CI[i] = dims_A[inds_C[i]]
        inds_CI[i] = inds_C[i]
        
    for i from 0 <= i < inds_I.shape[0]:
        dims_I[i] = dims_A[inds_I[i]]
        dims_CI[inds_C.shape[0] + i] = dims_A[inds_I[i]]
        inds_CI[inds_C.shape[0] + i] = inds_I[i]
    
    shape_C = 1
    for i from 0 <= i < inds_C.shape[0]:        
        shape_C = shape_C* dims_C[i]
        
    shape_I = 1
    for i from 0 <= i < inds_I.shape[0]:        
        shape_I = shape_I * dims_I[i]

    # Properties of the state to remain
    n = 0
    cdef int * inds_B = <int *> malloc((dims_A.shape[0] - inds_I.shape[0]-inds_C.shape[0]) * sizeof(int))
    for i from 0 <= i < dims_A.shape[0]:
        if not cy_in(i, inds_C) and not cy_in(i, inds_I):
            inds_B[n] = i
            n += 1
    
    cdef np.ndarray[ np.int32_t, mode="c"]  dims_B = np.zeros(n, dtype = np.int32)
    shape_B = 1
    for i from 0 <= i < dims_B.shape[0]:
        dims_B[i] = dims_A[inds_B[i]]
        shape_B = shape_B * dims_B[i]
        
    cdef np.ndarray[ complex, ndim=2, mode="c"] data_B = np.zeros([shape_B, shape_B], dtype=np.complex128)
    
    # Map to convert a multyindex the to the index of the matrix
    cdef int[::1] unrav_map_A = cy_unrav_map(dims_A)
    
    cdef complex[:,::1] coefs_list = np.ones([shape_C,shape_C], dtype = complex)
    cdef int[:,::1] inds_A = np.zeros([shape_I,shape_C], dtype = np.int32)
    cy_get_ellements_3(dims_I, inds_I, dims_C, inds_C, C_list, unrav_map_A, 
                                            shape_C, shape_I, coefs_list, inds_A)
    
    # To get other elmenets of B we shift all indices to a required constant.
    cdef int[:,:,::1] inds_A_list = np.empty([shape_B,len(inds_A), len(inds_A[0])], dtype = np.int32)
#     cdef int[::1] B_multind = np.empty(inds_B.shape[0], dtype = np.int32)
    cdef int *B_multind = <int *> malloc(dims_B.shape[0] * sizeof(int))
    
    for ind_B from 0 <= ind_B < shape_B:
        cy_unravel_inds( ind_B, dims_B, B_multind )
        
        ind_A = 0
        for i from 0 <= i < dims_B.shape[0]:
            ind_A += B_multind[i]*unrav_map_A[inds_B[i]]
        
        for i from 0 <= i < inds_A.shape[0]:
            for j from 0 <= j < inds_A.shape[1]:
                inds_A_list[ind_B,i,j] = inds_A[i,j] + ind_A
    
    cdef int[::1] y_inds_A, x_A_list
    # Calculation of indices for upper triangle + diagonal
    for y_B from 0 <= y_B < shape_B:
        for i from 0 <= i < inds_A.shape[0]:
            y_inds_A = inds_A_list[y_B][i]
            
            for x_B from y_B <= x_B < shape_B:
                for i_y_A from 0 <= i_y_A < y_inds_A.shape[0]:
                    y_A = y_inds_A[i_y_A]
                    x_A_list = inds_A_list[x_B][i]
                    for i_x_A from 0 <= i_x_A < x_A_list.shape[0]:
                        x_A = x_A_list[i_x_A]
                        data_B[y_B, x_B] = data_B[y_B, x_B] + A[y_A, x_A] * coefs_list[i_y_A, i_x_A]
                        
#       Calculation of indices for lower triangle
    for y_B from 0 <= y_B < shape_B:
        for x_B from y_B <= x_B < shape_B:
            data_B[x_B, y_B] = conj(data_B[y_B, x_B])
              
    return data_B, dims_B

## Example

In [16]:
dims_A = [10,2,2,2,2,2,2]
# dims_A = [2,2,2]
inds_I = [0]
inds_C = [1]
dims_C = [dims_A[i] for i in inds_C]
dims_I = [dims_A[i] for i in inds_I]



inds_B = [i for i in range(len(dims_A)) if i not in inds_I + inds_C]
rho_A = rand_dm(np.prod(dims_A), dims = [dims_A]*2)
A = rho_A.full()

C_full_list = [identity(dim) if i not in inds_C else rand_dm(dim) for i, dim in enumerate(dims_A)]
C_full = tensor(C_full_list)

C_list = [C_full_list[i] for i in inds_C]

C_list_arr = np.empty([len(C_list)]+list(C_list[0].shape), dtype = complex)
for i in range(len(C_list_arr)):
    C_list_arr[i] = C_list[i].full()
    
dims_A_ar = np.array(dims_A, dtype = np.int32)
dims_I_ar = np.array(dims_I, dtype = np.int32)
dims_C_ar = np.array(dims_C, dtype = np.int32)
inds_I_ar = np.array(inds_I, dtype = np.int32)
inds_C_ar = np.array(inds_C, dtype = np.int32)
data_B, dims_B = cy_contract(A, dims_A_ar, inds_I_ar, C_list_arr, inds_C_ar)
np.linalg.norm(data_B - (rho_A*C_full).ptrace(inds_B).full())

0.0

# Parallel

In [ ]:
%%cython
cimport cython
import numpy as np
cimport numpy as np
from cython.parallel import prange
cdef extern from "complex.h":
    double complex conj(double complex x)
    
@cython.boundscheck(False)
@cython.cdivision(True)        
cdef void cy_unravel_inds( int ind, int[::1] dims, int * multy_inds ) nogil:
    
    cdef size_t j
    for j from dims.shape[0] > j >= 0:
        
        multy_inds[j] = ind % dims[j]
        ind = ind / dims[j]

cdef int[::1] cy_unrav_map(int[::1] dims):
    cdef int[::1] map_ar = np.ones(dims.shape[0], dtype=np.int32)
    
    cdef size_t j
    for j from dims.shape[0] > j >= 1:
        map_ar[j-1] = map_ar[j]*dims[j]
    
    return map_ar

@cython.boundscheck(False)
@cython.wraparound(False)
cdef int cy_in(int val, int[::1] vec) nogil:
    # val in vec in pure cython
    cdef size_t ii
    for ii from 0 <= ii < vec.shape[0]:
        if val == vec[ii]:
            return 1
    return 0

@cython.boundscheck(False)
@cython.wraparound(False)
cdef void cy_get_ellements_3(int *dims_I, int[::1] inds_I, int *dims_C, int[::1] inds_C, 
                      complex[:,:,::1] C_list, int[::1] unrav_map_A, size_t shape_C, size_t shape_I, 
                       complex[:,::1] coefs_list, int[:,::1] inds_A) nogil:
    
#     cdef size_t n_x, n_A, D, i, k, k1, k2, j, m, c1, c2, n_0, n_1 
#     cdef int d_dim
    cdef size_t n_x, n_A, D, i, k, k1, k2, j, m, c1, c2, n_0, n_1 
    cdef int d_dim
    
    n_x = 1
    n_A = 1
    for i from 0 <= i < inds_C.shape[0]:
        D = dims_C[i]
        d_dim = unrav_map_A[inds_C[i]]
        for k from 1 <= k < D:
            n_0 = (k-1)*n_A
            n_1 = k*n_A
            for j from 0 <= j < n_A:
                inds_A[0][n_1 + j] = inds_A[0][n_0 + j] + d_dim
        
        n_A = n_A*D

        for k1 from D > k1 >= 0:
            for c1 from n_x > c1 >= 0:
                for c2 from n_x > c2 >= 0:
                    for k2 from D > k2 >= 0:
                        coefs_list[k1*n_x+c1][k2*n_x + c2] = coefs_list[c1][c2] * C_list[i, k2, k1]
                        
        n_x = D*n_x
        
    n_A = 1
    n_1 = 1
    for i from 0 <= i < inds_I.shape[0]:
        D = dims_I[i]
        for k from 1 <= k < D:
            d_dim = k*unrav_map_A[inds_I[i]]
            for j from 0 <= j < n_A:
                for m from 0 <= m < shape_C:
                    inds_A[n_1][m] = inds_A[j][m] + d_dim
                n_1 += 1
        n_A = n_1

from libc.stdlib cimport malloc, free
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef cy_contract_parallel(complex[:,::1] A, int[::1] dims_A, int[::1] inds_I, complex[:,:,::1] C_list, int[::1] inds_C):
    
    '''
    Partial trace of indices inds_I + contraction of inds_C of A with operators in C_list
    '''
    
    cdef size_t shape_A, shape_B, shape_C, shape_I, i, j, y_B, x_B, i_y_A, i_x_A, n, ind_A, y_A, x_A
    
    # Shape of A
    shape_A = A.shape[0]
    cdef int *dims_C = <int *> malloc(inds_C.shape[0] * sizeof(int))
    cdef int *dims_I = <int *> malloc(inds_I.shape[0] * sizeof(int))
    cdef int *dims_CI = <int *> malloc((inds_C.shape[0] + inds_I.shape[0]) * sizeof(int))
    cdef int *inds_CI = <int *> malloc((inds_C.shape[0] + inds_I.shape[0]) * sizeof(int))

    for i from 0 <= i < inds_C.shape[0]:
        dims_C[i] = dims_A[inds_C[i]]
        dims_CI[i] = dims_A[inds_C[i]]
        inds_CI[i] = inds_C[i]
        
    for i from 0 <= i < inds_I.shape[0]:
        dims_I[i] = dims_A[inds_I[i]]
        dims_CI[inds_C.shape[0] + i] = dims_A[inds_I[i]]
        inds_CI[inds_C.shape[0] + i] = inds_I[i]
    
    shape_C = 1
    for i from 0 <= i < inds_C.shape[0]:        
        shape_C = shape_C* dims_C[i]
        
    shape_I = 1
    for i from 0 <= i < inds_I.shape[0]:        
        shape_I = shape_I * dims_I[i]

    # Properties of the state to remain
    n = 0
    cdef int * inds_B = <int *> malloc((dims_A.shape[0] - inds_I.shape[0]-inds_C.shape[0]) * sizeof(int))
    for i from 0 <= i < dims_A.shape[0]:
        if not cy_in(i, inds_C) and not cy_in(i, inds_I):
            inds_B[n] = i
            n += 1
    
    cdef np.ndarray[ np.int32_t, mode="c"]  dims_B = np.zeros(n, dtype = np.int32)
    shape_B = 1
    for i from 0 <= i < dims_B.shape[0]:
        dims_B[i] = dims_A[inds_B[i]]
        shape_B = shape_B * dims_B[i]
        
    cdef np.ndarray[ complex, ndim=2, mode="c"] data_B = np.zeros([shape_B, shape_B], dtype=np.complex128)
    
    # Map to convert a multyindex the to the index of the matrix
    cdef int[::1] unrav_map_A = cy_unrav_map(dims_A)
    
    cdef complex[:,::1] coefs_list = np.ones([shape_C,shape_C], dtype = complex)
    cdef int[:,::1] inds_A = np.zeros([shape_I,shape_C], dtype = np.int32)
    cy_get_ellements_3(dims_I, inds_I, dims_C, inds_C, C_list, unrav_map_A, 
                                            shape_C, shape_I, coefs_list, inds_A)
    
    # To get other elmenets of B we shift all indices to a required constant.
    cdef int[:,:,::1] inds_A_list = np.empty([shape_B,len(inds_A), len(inds_A[0])], dtype = np.int32)
#     cdef int[::1] B_multind = np.empty(inds_B.shape[0], dtype = np.int32)
    cdef int *B_multind = <int *> malloc(dims_B.shape[0] * sizeof(int))
    
    for ind_B from 0 <= ind_B < shape_B:
        cy_unravel_inds( ind_B, dims_B, B_multind )
        
        ind_A = 0
        for i from 0 <= i < dims_B.shape[0]:
            ind_A += B_multind[i]*unrav_map_A[inds_B[i]]
        
        for i from 0 <= i < inds_A.shape[0]:
            for j from 0 <= j < inds_A.shape[1]:
                inds_A_list[ind_B,i,j] = inds_A[i,j] + ind_A
    
    cdef int [:, :, ::1] inds_A_list_view = inds_A_list
    cdef int[::1] y_inds_A, x_A_list
    # Calculation of indices for upper triangle + diagonal
    for y_B in prange(shape_B, nogil=True):
#     for y_B from 0 <= y_B < shape_B:
        for i from 0 <= i < inds_A.shape[0]:
            
            for x_B from y_B <= x_B < shape_B:
                for i_y_A from 0 <= i_y_A < inds_A_list_view[y_B][i].shape[0]:
                    y_A = inds_A_list_view[y_B][i][i_y_A]
                    for i_x_A from 0 <= i_x_A < inds_A_list[x_B][i].shape[0]:
                        x_A = inds_A_list[x_B][i][i_x_A]
                        data_B[y_B, x_B] = data_B[y_B, x_B] + A[y_A, x_A] * coefs_list[i_y_A, i_x_A]
                        
#       Calculation of indices for lower triangle
    for y_B from 0 <= y_B < shape_B:
        for x_B from y_B <= x_B < shape_B:
            data_B[x_B, y_B] = conj(data_B[y_B, x_B])
              
    return data_B, dims_B